In [ ]:
# csv 转化为 json

import pandas as pd
import json

items = {}
item_df = pd.read_csv('Beauty2014/Beauty2014.item.csv')
for row in item_df.itertuples():
    item_id = getattr(row, "item_id")
    title = getattr(row, "title")
    discription = getattr(row, "description")
    title = "" if pd.isnull(title) else title
    discription = "" if pd.isnull(discription) else discription
    items[item_id] = {"title": title, "discription": discription}

with open('Beauty2014/Beauty2014.item.json', 'w') as f:
    json.dump(items, f, ensure_ascii=False, indent=4)

inters = {}
inter_df = pd.read_csv("Beauty2014/Beauty2014.inter.csv")
for row in inter_df.itertuples():
    user_id = getattr(row, "user_id")
    item_id = getattr(row, "item_id")
    if user_id not in inters:
        inters[user_id] = []
    inters[user_id].append(item_id)

with open('Beauty2014/Beauty2014.inter.json', 'w') as f:
    json.dump(inters, f, ensure_ascii=False, indent=4)


In [ ]:
# 统计数据集的信息
import json

inters = json.load(open("Beauty2014/Beauty2014.inter.json", "r"))

user_count = {}
item_count = {}

for user_id, items in inters.items():
    for item_id in items:
        if item_id not in item_count:
            item_count[item_id] = 0
        item_count[item_id] += 1
    user_count[user_id] = len(items)

train_target_count = []
valid_target_count = []
test_target_count = []

for user_id, items in inters.items():
    for index in range(1, len(items)-2):
        train_target_count.append(item_count[items[index]])
    valid_target_count.append(item_count[items[-2]])
    test_target_count.append(item_count[items[-1]])
print(f"item_avg_count: {sum(item_count.values())/len(item_count)}")
print(f"train_target_count: {sum(train_target_count)/len(train_target_count)}")
print(f"valid_target_count: {sum(valid_target_count)/len(valid_target_count)}")
print(f"test_target_count: {sum(test_target_count)/len(test_target_count)}")

item_avg_count: 16.403768283612923
train_target_count: 53.21257409845297
valid_target_count: 45.58297187318338
test_target_count: 40.97030809819792


In [ ]:
# 将sasrec.pth 转化为 pt
import torch
import os
path = "" # dir path
os.chdir(path)  # 切换目录
sasrec = torch.load("save/Beauty2014/SASRec-d_model_32-train_batch_size_256-lr_0.01-wd_0.01.pth", weights_only=False)
emb = sasrec.item_emb.weight
torch.save(emb, "data/Beauty2014/Beauty2014-cf_emb.pt")